	# Author: Alexander Staub
	## Last changed: 2025.02.04
	## Purpose: Trialing the Chartmetrci API


In [1]:
#installing packages
import requests
from pprint import pprint

In [9]:
# Define API host and your refresh token
HOST = 'https://api.chartmetric.com'
with open("chartmetric_refresh_token.txt", "r") as f:
    REFRESH_TOKEN = f.read().strip()

In [10]:
# Retrieve an access token using the refresh token
token_response = requests.post(f'{HOST}/api/token', json={'refreshtoken': REFRESH_TOKEN})
if token_response.status_code != 200:
    raise Exception(f"Error: received {token_response.status_code} from /api/token")
access_token = token_response.json()['token']

In [11]:
# Setup headers with the retrieved access token
headers = {'Authorization': f'Bearer {access_token}'}

def get_request(endpoint, params=None, retry=True):
    response = requests.get(f"{HOST}{endpoint}", headers=headers, params=params)
    if response.status_code == 401 and retry:
        # Refresh the token
        token_response = requests.post(f'{HOST}/api/token', json={'refreshtoken': REFRESH_TOKEN})
        if token_response.status_code != 200:
            raise Exception(f"Error: received {token_response.status_code} when refreshing token")
        new_token = token_response.json()['token']
        headers['Authorization'] = f'Bearer {new_token}'
        # Retry the request once
        response = requests.get(f"{HOST}{endpoint}", headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"Error: received {response.status_code} when accessing {endpoint} - {response.text}")
    return response.json()

Use Spotify ID to access song characteristics:
- load in a subset of the spotify songs with spotify IDs
- Loop over them to get the chartmetric IDs and any other further information accessible
- use chartmetric IDs to get the information of relevance
- setup code in a way that allows us to get information for our relevant songs

In [6]:
#read in the first 100 lines from the file "AD_spotify_accoustic_char_250k.csv" in the data/raw_data/Spotify folder and call it spotify_sample
# the data directory is 2 directories down from the current directory
# it should be a pandas dataframe and the csv has headers

import pandas as pd

spotify_sample = pd.read_csv("../../data/raw_data/Spotify/AD_spotify_accoustic_char_250k.csv", nrows=100)


In [12]:
# Define a function that retrieves Chartmetric IDs for a given ISRC code

def get_chartmetric_ids(isrc):

    # Define the endpoint for the API request
    endpoint = f"/api/track/isrc/{isrc}/get-ids"

    # Make the API request
    try:
        response = get_request(endpoint)

    # Catch any exceptions that occur during the request
    except Exception as e:
        # Log the error if needed; return NA (using None) and proceed
        print(f"Error for ISRC {isrc}: {e}")
        return None

    # Check if a valid object was returned; if not, return NA
    if response.get("obj") and isinstance(response["obj"], list) and len(response["obj"]) > 0:
        return response["obj"][0].get("chartmetric_ids", None)
    else:
        return None

# Apply the function to the ISRC column in the Spotify sample data
spotify_sample["chartmetric_ids"] = spotify_sample["ISRC"].apply(get_chartmetric_ids)



In [13]:
#transform the object type into float 
spotify_sample["chartmetric_ids"] = spotify_sample["chartmetric_ids"].apply(
    lambda x: float(x[0]) if isinstance(x, list) and len(x) > 0 else None
)

In [14]:
#retain only the rows with unique chartmetric_ids
spotify_sample_unique = spotify_sample.drop_duplicates(subset="chartmetric_ids")

In [ ]:
# try the playlist endpoint with an actual playlist id = 2183278

playlist_endpoint = '/api/playlist/spotify/2183278/current/tracks'

playlist_output = get_request(playlist_endpoint)

pprint(playlist_output)